In [ ]:
!pip install -U transformers

# 数据收集和同一格式

In [ ]:
from datasets import load_dataset


import random


# 加载 Belle 1M 中文数据集
dataset = load_dataset("BelleGroup/train_1M_CN", split="train")
sample_size = 10000
sampled = random.sample(list(dataset), sample_size)

print(f"通用数据抽取条数: {len(sampled)}")
print(f"示例{sampled[0]}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/941 [00:00<?, ?B/s]

Belle_open_source_1M.json:   0%|          | 0.00/458M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/917424 [00:00<?, ? examples/s]

通用数据抽取条数: 10000
示例{'instruction': '将以下一段话翻译成日语：“我喜欢吃咸味的食物，如薯片和肉干。”\n', 'input': '', 'output': '「私は、ポテトチップスやビーフジャーキーなどの塩味のある食べ物が好きです。」'}


In [ ]:
import json
custom_data_path = "/content/hit.json"  # 哈工大数据文件路径
with open(custom_data_path, "r", encoding="utf-8") as f:
    custom_data = json.load(f)

print(f"自建数据条数: {len(custom_data)}")
print(f"示例{custom_data[0]}")


自建数据条数: 611
示例{'system': '你是一个了解哈工大历史的助手', 'user': '哈工大的建校时间和最初校名是什么？', 'assistant': '哈工大始建于1920年，最初的校名为“哈尔滨中俄工业学校”，是中国近代培养工业技术人才的重要源头之一。'}


In [ ]:
from tqdm import tqdm
def convert_to_prompt(system_text, user_text, assistant_text):
    """生成符合实验要求的完整prompt字符串"""
    return (
        f"<|beginofutterance|>系统\n{system_text}\n<|endofutterance|>\n"
        f"<|beginofutterance|>用户\n{user_text}\n<|endofutterance|>\n"
        f"<|beginofutterance|>智能助手\n{assistant_text}\n<|endofutterance|>"
    )

formatted_data = []

# ----------  处理自建数据 ----------
for item in tqdm(custom_data, desc="格式化自建数据"):
    system = item.get("system", "你是一个乐于助人的中文智能助手。")
    user = item.get("user", "")
    assistant = item.get("assistant", "")
    formatted_data.append({
        "prompt": convert_to_prompt(system, user, assistant),
    })

# ----------  处理通用数据 ----------
for item in tqdm(sampled, desc="格式化通用数据"):
    instruction = item.get("instruction", "")
    input_text = item.get("input", "")
    output_text = item.get("output", "")

    # 拼接用户提问
    if input_text:
        user = f"{instruction}\n{input_text}"
    else:
        user = instruction

    formatted_data.append({
        "prompt": convert_to_prompt("你是一个乐于助人的中文智能助手。", user, output_text),
    })

print(f"最终合并数据条数: {len(formatted_data)}")
print(f"示例{formatted_data[0]},\n{formatted_data[-1]}")

格式化通用数据: 100%|██████████| 10000/10000 [00:00<00:00, 332409.04it/s]

最终合并数据条数: 10611
示例{'prompt': '<|beginofutterance|>系统\n你是一个了解哈工大历史的助手\n<|endofutterance|>\n<|beginofutterance|>用户\n哈工大的建校时间和最初校名是什么？\n<|endofutterance|>\n<|beginofutterance|>智能助手\n哈工大始建于1920年，最初的校名为“哈尔滨中俄工业学校”，是中国近代培养工业技术人才的重要源头之一。\n<|endofutterance|>'},
{'prompt': '<|beginofutterance|>系统\n你是一个乐于助人的中文智能助手。\n<|endofutterance|>\n<|beginofutterance|>用户\n为一篇新闻文章生成一个有吸引力的标题。\n文本：数百只鲨鱼聚集在南非海岸，当地居民被迫撤离。\n<|endofutterance|>\n<|beginofutterance|>智能助手\n南非海岸上出现数百只鲨鱼，当地居民急速疏散\n<|endofutterance|>'}


In [ ]:
output_path = "/content/sft_data_merged.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for item in formatted_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"已保存到 {output_path}")

from google.colab import files


files.download(output_path)

已保存到 /content/sft_data_merged.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 加载base模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
)
# 设置padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.add_special_tokens({
    "additional_special_tokens": [
        "<|beginofutterance|>",
        "<|endofutterance|>"
    ]
})
model.resize_token_embeddings(len(tokenizer))

special_tokens = ["<|beginofutterance|>", "<|endofutterance|>"]

for t in special_tokens:
    ids = tokenizer.encode(t, add_special_tokens=False)
    print(f"{t} -> {ids} -> {tokenizer.decode(ids)}")

# 简单调用
prompt = "请介绍哈尔滨工业大学"
messages = [
    {"role": "system", "content": "你是Qwen2.5，一个通用领域大模型"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(f"response:\n{response}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

<|beginofutterance|> -> [151665] -> <|beginofutterance|>
<|endofutterance|> -> [151666] -> <|endofutterance|>
response:
作为人工智能技术的代表，我并没有实际的身份或知识来源。因此，无法提供关于哈尔滨工业大学的具体信息。如果您有其他问题需要帮助，请告诉我，我会尽力回答。


# 加载数据集

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/sft_data_merged.jsonl", split="train")

print(f"data:\n{dataset[0]}")

Generating train split: 0 examples [00:00, ? examples/s]

data:
{'prompt': '<|beginofutterance|>系统\n你是一个了解哈工大历史的助手\n<|endofutterance|>\n<|beginofutterance|>用户\n哈工大的建校时间和最初校名是什么？\n<|endofutterance|>\n<|beginofutterance|>智能助手\n哈工大始建于1920年，最初的校名为“哈尔滨中俄工业学校”，是中国近代培养工业技术人才的重要源头之一。\n<|endofutterance|>'}


In [ ]:
def preprocess(example):
    """
    将你的数据集拆分为输入和标签
    """
    # example['prompt'] 包含 "<|beginofutterance|>系统...<|endofutterance|><|beginofutterance|>用户..."
    # 把 answer 提取出来作为 labels
    text = example["prompt"]

    # 找到智能助手标记的位置
    assistant_start = text.find("<|beginofutterance|>智能助手")
    if assistant_start == -1:
        raise ValueError("找不到智能助手标记")

    # 输入部分
    input_text = text[:assistant_start + len("<|beginofutterance|>智能助手\n")]

    # 输出部分（answer + 结束标记）
    answer_text = text[assistant_start + len("<|beginofutterance|>智能助手\n"):]

    # 拼接 tokenizer
    input_ids = tokenizer(input_text, truncation=True, max_length=1024).input_ids
    labels = tokenizer(answer_text, truncation=True, max_length=1024).input_ids

    # 只计算 answer 的 loss，把 input 部分的 label 置为 -100
    labels = [-100]*len(input_ids) + labels

    return {"input_ids": input_ids + labels[len(input_ids):], "labels": labels}

tokenized_dataset = dataset.map(preprocess, remove_columns=["prompt"])

Map:   0%|          | 0/10611 [00:00<?, ? examples/s]

# 配置LoRA


In [ ]:
from peft import LoraConfig, get_peft_model
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1)

model = get_peft_model(model, peft_config)
# 打印可训练参数
model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,332,416 || trainable%: 0.1094


# 配置SFTTrainer训练参数

In [ ]:
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 12.5 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTConfig

output_dir = "/content/sft_output"

args = SFTConfig(
    output_dir=output_dir,
      per_device_train_batch_size=2,
      gradient_accumulation_steps=4,
      learning_rate=2e-5,
      num_train_epochs=3,
      logging_steps=50,
      save_strategy="epoch",
      fp16=True,
      report_to="none",
      packing=True,
      max_length=512,

)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 训练

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=args,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Packing train dataset:   0%|          | 0/10611 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
50,2.988900
100,2.670100
150,2.402700
200,2.171100
250,2.115100
300,2.036900
350,2.040400
400,1.994500
450,1.985700
500,1.985300


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=972, training_loss=2.099210005238223, metrics={'train_runtime': 1656.299, 'train_samples_per_second': 4.693, 'train_steps_per_second': 0.587, 'total_flos': 8459586862695936.0, 'train_loss': 2.099210005238223, 'entropy': 1.927680809389461, 'num_tokens': 3933534.0, 'mean_token_accuracy': 0.5985497666353529, 'epoch': 3.0})

# 测试

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 基础模型路径
base_model_path = "Qwen/Qwen2.5-0.5B-Instruct"

# LoRA checkpoint
peft_model_path = "/content/drive/MyDrive/sft_output/checkpoint-972"

# 1. 加载基础模型
model = AutoModelForCausalLM.from_pretrained(base_model_path)

# 2. 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

# 3. 加载 LoRA 权重

model = PeftModel.from_pretrained(model, peft_model_path, ignore_mismatched_sizes=True)



In [ ]:
import torch


system_prompt = "你是一个了解哈工大历史的中文助手。"
user_query = "请简要介绍哈工大的专业"

prompt = f"""
<|beginofutterance|>系统
{system_prompt}
<|endofutterance|>
<|beginofutterance|>用户
{user_query}
<|endofutterance|>
<|beginofutterance|>智能助手

"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("\n================ 模型完整输出 ================\n")
print(decoded)
print("\n================ 回答部分 ================\n")
if "<|beginofutterance|>智能助手" in decoded:
    print(decoded.split("<|beginofutterance|>智能助手")[-1].strip())
else:
    print("未检测到回答，请检查prompt格式。")



================ 模型完整输出 ================


<|beginofutterance|>系统
你是一个了解哈工大历史的中文助手。
<|endofutterance|>
<|beginofutterance|>用户
请简要介绍哈工大的专业
<|endofutterance|>
<|beginofutterance|>智能助手

哈工大是中华人民共和国教育部直属重点综合性大学，创建于1958年，位于中国北京。学校具有丰富的科研和教学资源、高水平的人才培养体系以及良好的国际化办学基础，在工科领域拥有较强的综合实力。

哈工大设有4个学院，分别是机械工程学院、计算机科学与技术学院、材料科学与工程学院、化工与生物技术学院；下设60多个二级学科门类，涵盖应用数学、自动化、控制科学与工程等众多前沿科技领域。学校还设有23个本科专业，涵盖工学、理学、管理学、文学、医学、艺术、教育学等多个学科门类。

哈工大在工程技术、机械制造、电子电器、信息通信等领域均具备较强的优势和实力，近年来在国内外学术界和工业界享有盛誉。

哈工大致力于培养高素质的技术创新人才和技术领军人才，并在全球范围内开展国际合作与交流，为国家和社会发展作出贡献。

哈佛大学的历史可以追溯到16世纪，其创始人之一罗伯特·波普尔（Robert Peel）出生于美国新泽西州的斯克里普斯，他在17世纪初建立了哈佛大学，成为美国最古老的大学之一。哈佛大学是

================ 回答部分 ================

哈工大是中华人民共和国教育部直属重点综合性大学，创建于1958年，位于中国北京。学校具有丰富的科研和教学资源、高水平的人才培养体系以及良好的国际化办学基础，在工科领域拥有较强的综合实力。

哈工大设有4个学院，分别是机械工程学院、计算机科学与技术学院、材料科学与工程学院、化工与生物技术学院；下设60多个二级学科门类，涵盖应用数学、自动化、控制科学与工程等众多前沿科技领域。学校还设有23个本科专业，涵盖工学、理学、管理学、文学、医学、艺术、教育学等多个学科门类。

哈工大在工程技术、机械制造、电子电器、信息通信等领域均具备较强的优势和实力，近年来在国内外学术界和工业界享有盛誉。

哈工大致力于培养高素质的技术创新人才和技术领军人才，并在全球范围内开展国际合

# 保存

In [13]:
merged_model = model.merge_and_unload()
# merged_model.save_pretrained("/content/merged_model")
tokenizer.save_pretrained("/content//merged_model")

('/content//merged_model/tokenizer_config.json',
 '/content//merged_model/special_tokens_map.json',
 '/content//merged_model/chat_template.jinja',
 '/content//merged_model/vocab.json',
 '/content//merged_model/merges.txt',
 '/content//merged_model/added_tokens.json',
 '/content//merged_model/tokenizer.json')